In [1]:
import numpy as np

from itertools import product
from collections import deque

In [2]:
DIRECTIONS = [(-1,0), (1,0), (0,-1), (0,1)]

In [3]:
with open('data/12-12.input') as f:
    data = f.read().strip()

In [4]:
plants = np.array([list(x) for x in data.split('\n')])

visited = np.array([[False for col in range(plants.shape[1])] for row in range(plants.shape[0])])

In [5]:
def find_regions(p):
    visited = np.array([[False for col in range(p.shape[1])] for row in range(p.shape[0])])
    all_regions = []
    while not np.all(visited):
        #  Find a starting point to search for an area of like plants
        for r,c in product(range(p.shape[0]), range(p.shape[1])):
            if not visited[r, c]:
                start = (r,c)
                break
        #  Then do a search (basically a BFS) to find plants of this plant type 
        #  that create a connected region.  This could contain "holes".
        plant_type = p[r,c]
        queue = deque([(r,c)])
        visited[r, c] = True
        region = [(r,c)]
        while queue:
            curr_x, curr_y = queue.popleft()
            for dx, dy in DIRECTIONS:
                test_x, test_y = (curr_x + dx, curr_y + dy)
                if (0 <= test_x < p.shape[0]) and (0 <= test_y < p.shape[1])\
                        and (p[test_x, test_y] == plant_type)\
                        and (not visited[test_x, test_y]):
                    visited[test_x, test_y] = True
                    queue.append((test_x, test_y))
                    region.append((test_x, test_y))
        all_regions.append(region)
        
    return all_regions

In [6]:
def area(nodes):
    #  The area is just the number of nodes in the (connected) region.
    return len(nodes)

def perimeter(nodes):
    #  Finding the perimeter is relatively easy.  
    #  Go through the nodes and check if the four possible neighbors of each node
    #  are included in the given set of nodes.  If not, you have found an edge of 
    #  that is included in the perimeter of the region (whether it be an "external"
    #  edge, or an edge of a subregion of a different type).  
    perimeter = 0
    for n_x, n_y in nodes:
        for dx, dy in DIRECTIONS:
            test = (n_x + dx, n_y + dy)
            if test not in nodes:
                perimeter = perimeter + 1
                
    return perimeter

def cost(nodes): 
    return area(nodes) * perimeter(nodes)

## Part I

In [7]:
print(f'Total cost using perimeter: {sum(cost(r) for r in find_regions(plants))}')

Total cost using perimeter: 1485656


## Part II

This method is a little convoluted, and you have to account for an example like the one given in the problem description that looks like this (where each interior "B" region has four sides, so the total number of sides from the "A" region is 12):
AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA

We first find the individual "pieces" that would consist of each edge, then we "glue" the edges together in order to find the edges that bound the region.  Finally, we return the number of such edges from this method, as this is what is relevant in terms of finding the "cost" of the region.  

In [8]:
def bounding_edges(nodes):
    pieces = []
    for n_x, n_y in nodes:
        for d, offset in zip(DIRECTIONS, [(-0.25,0), (0.25,0), (0,-0.25), (0,0.25)]):
            test = (n_x + d[0], n_y + d[1])
            if test not in nodes:
                pieces.append((n_x + offset[0], n_y + offset[1]))

    pieces = list(set(pieces))
    verticals = sorted([e for e in pieces if e[0] % 1 in (0.25, 0.75)])
    horizontals = sorted([e for e in pieces if e not in verticals])
    
    b_v = []
    while verticals:
        first = verticals[0]
        edge = [first]
        curr = first
        while (curr[0], curr[1] + 1) in verticals:
            curr = (curr[0], curr[1] + 1)
            edge.append(curr)
        curr = first
        while (curr[0], curr[1] - 1) in verticals:
            curr = (curr[0], curr[1] - 1)
            edge.append(curr)
        b_v.append(edge)
        verticals = [e for e in verticals if e not in edge]
            
    b_h = []
    while horizontals:
        first = horizontals[0]
        edge = [first]
        curr = first
        while (curr[0] + 1, curr[1]) in horizontals:
            curr = (curr[0] + 1, curr[1])
            edge.append(curr)
        curr = first
        while (curr[0] - 1, curr[1]) in horizontals:
            curr = (curr[0] - 1, curr[1])
            edge.append(curr)
        b_h.append(edge)
        horizontals = [e for e in horizontals if e not in edge]
   
    return len(b_v) + len(b_h)

In [9]:
def cost(region):
    return area(region) * bounding_edges(region)

In [10]:
sum(cost(r) for r in find_regions(plants))

899196